<a href="https://colab.research.google.com/github/srdiegorodrigues/projeto/blob/main/Por_rodovia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Importação das bibliotecas
import pandas as pd
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import io

In [4]:
#Carregar arquivos a serem analisados
url = "/content/drive/MyDrive/MESTRADO/DADOS/OCORRENCIAS/"

df_2017 = pd.read_csv(io.StringIO(open(f'{url}datatran2017.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2018 = pd.read_csv(io.StringIO(open(f'{url}datatran2018.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2019 = pd.read_csv(io.StringIO(open(f'{url}datatran2019.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2020 = pd.read_csv(io.StringIO(open(f'{url}datatran2020.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2021 = pd.read_csv(io.StringIO(open(f'{url}datatran2021.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2022 = pd.read_csv(io.StringIO(open(f'{url}datatran2022.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)

df_2017 = df_2017.dropna()
df_2018 = df_2018.dropna()
df_2019 = df_2019.dropna()
df_2020 = df_2020.dropna()
df_2021 = df_2021.dropna()
df_2022 = df_2022.dropna()

dfs = [df_2017, df_2018, df_2019, df_2020, df_2021, df_2022]

<ipython-input-4-5849736c2be4>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_2017 = pd.read_csv(io.StringIO(open(f'{url}datatran2017.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
<ipython-input-4-5849736c2be4>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_2018 = pd.read_csv(io.StringIO(open(f'{url}datatran2018.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
<ipython-input-4-5849736c2be4>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_2019 = pd.read_csv(io.StringIO(open(f'{url}datatran2019.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
<ipython-input-4-5849736c2be4>:7: FutureWarning: The error_bad_lines a

In [5]:
from sklearn.preprocessing import MinMaxScaler


#carregar base de dados dos feriados nacionais
df_feriado = pd.read_csv("/content/drive/MyDrive/MESTRADO/DADOS/OCORRENCIAS/feriados_nacionais.csv",  encoding='ISO-8859-1', sep=";")

#arredondar o valor de km para cima ou para baixo, conforme o valor após o ponto
def arredondar_km(valor):
    valor_float = float(valor)  # Converter para float
    inteiro = int(valor_float)  # parte inteira do valor
    decimal = valor_float - inteiro  # parte decimal do valor

    # Se o valor decimal for maior ou igual a 0.5, arredonda para cima, caso contrário, arredonda para baixo
    if decimal >= 0.5:
        return inteiro + 1
    else:
        return inteiro

#converter o valor da feature br para int
def br_int(valor):
    valor_float = float(valor)
    valor_int = int(valor_float)
    return valor_int

dias_semana_para_int = {
        'segunda-feira': 0,
        'terça-feira': 1,
        'quarta-feira': 2,
        'quinta-feira': 3,
        'sexta-feira': 4,
        'sábado': 5,
        'domingo': 6
    }

def converter_dia_semana_para_int(dia):
    return dias_semana_para_int[dia]

for df in dfs:
    #alterar condições metereologicas para evitar outliers
    df['condicao_metereologica'] = df['condicao_metereologica'].str.replace('Ignorado', 'Céu Claro')
    df['condicao_metereologica'] = df['condicao_metereologica'].str.replace('Neve', 'Chuva')

    #substituir / por - na data_inversa
    df['data_inversa'] = df['data_inversa'].str.replace('/', '-')

    #criação da feature chuva para determinar se a pista estava molhada na hora do acidente
    for indice, item in df.iterrows():
        if item['condicao_metereologica'] in ['Chuva', 'Garoa/Chuvisco', 'Granizo']:
            df.at[indice, 'chuva'] = 1
        else:
            df.at[indice, 'chuva'] = 0

    #fragmentação da feature data_inversa em dia, mes e ano
    df[['dia', 'mes', 'ano']] = df['data_inversa'].apply(lambda x: pd.Series(x.split('-')).astype(int))

    # #Concatenar colunas data_inversa e horario e criando a feature data_hora
    df['data_hora'] = df.apply(lambda x: '%s-%s' % (x['data_inversa'],x['horario']), axis=1)
    #Criação da feature data_hora_int
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    # Convertendo as datas para timestamps Unix
    df['data_hora_int'] = df['data_hora'].astype(int) // 10**9

    #normalizar a feature data_hora_int
    scaler = MinMaxScaler()
    df['data_hora_int'] = scaler.fit_transform(df['data_hora_int'].values.reshape(-1, 1))

    #substituir a , por . na feature km
    df['km'] = df['km'].str.replace(',','.')
    df['km'] = df['km'].apply(arredondar_km)
    df['br'] = df['br'].apply(br_int)


    df['dia_semana_int'] =  df['dia_semana'].apply(converter_dia_semana_para_int)

    #Setar se dia da ocorrência do acidente era feriado ou não
    #Criando a feature feriado com base nas portarias do governo federal
    #Caso seja feriado, será atribuido o valor 1 para o registro
    #caso contrário, será atribuido o valor 0
    df['data_inversa'] = pd.to_datetime(df['data_inversa'])
    df_feriado['Data'] = pd.to_datetime(df_feriado['Data'])
    df_feriado['Data'] = df_feriado['Data'].sort_values(ascending=False).values
    # Criando uma nova coluna 'feriado' no DataFrame principal e inicializando com 0
    df['feriado'] = 0

    # Loop for para comparar as datas e atribuir 1 se forem iguais
    for index, row in df.iterrows():
        if row['data_inversa'] in df_feriado['Data'].values:
            df.at[index, 'feriado'] = 1

<ipython-input-5-7b976e7aa780>:79: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['data_inversa'] = pd.to_datetime(df['data_inversa'])
<ipython-input-5-7b976e7aa780>:80: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_feriado['Data'] = pd.to_datetime(df_feriado['Data'])


#Divisão dos dados que serão utilizados nos modelos

In [6]:
#features que serão usadas para execução dos modelos

# Selecionar as colunas categóricas
from sklearn.preprocessing import LabelEncoder, StandardScaler

cols_categoricas = ['uf','condicao_metereologica','delegacia','uop']

# Aplicar a codificação LabelEncoder em cada coluna categórica
label_encoder = LabelEncoder()
for df in dfs:
    for col in cols_categoricas:
        df[col] = label_encoder.fit_transform(df[col])

for df in dfs:
    df['regional_int'] = label_encoder.fit_transform(df['regional'])

colunas = ['km', 'uf', 'br', 'condicao_metereologica', 'regional','regional_int', 'delegacia', 'uop','chuva','dia','mes','ano','feriado', 'dia_semana_int']

In [7]:
#base de dados para algoritmos de regressão e classificação
treinamento1 = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021], ignore_index=True) #TRAIN_RAW
teste1 = df_2022[colunas] #TEST_RAW

#dados para teste
treinamento1 = treinamento1[colunas]
sr_df_r = treinamento1[treinamento1['regional'] == 'SPRF-DF']
sr_df_br_020_r = sr_df_r[sr_df_r['br'] == 20]
sr_df_br_040_r = sr_df_r[sr_df_r['br'] == 40]
sr_sp_r = treinamento1[treinamento1['regional'] =='SPRF-SP']
sr_sp_br_101_r = sr_sp_r[sr_sp_r['br'] == 101]
sr_sp_br_116_r = sr_sp_r[sr_sp_r['br'] == 116]
sr_sc_r = treinamento1[treinamento1['regional'] =='SPRF-SC']
sr_sc_br_101_r = sr_sc_r[sr_sc_r['br'] == 101]
sr_sc_br_116_r = sr_sc_r[sr_sc_r['br'] == 116]

#dados para teste
sr_df_t_r = teste1[teste1['regional'] == 'SPRF-DF']
sr_df_br_020_t_r = sr_df_t_r[sr_df_t_r['br'] == 20]
sr_df_br_040_t_r = sr_df_t_r[sr_df_t_r['br'] == 40]
sr_sp_t_r = teste1[teste1['regional'] =='SPRF-SP']
sr_sp_br_101_t_r = sr_sp_t_r[sr_sp_t_r['br'] == 101]
sr_sp_br_116_t_r = sr_sp_t_r[sr_sp_t_r['br'] == 116]
sr_sc_t_r = teste1[teste1['regional'] =='SPRF-SC']
sr_sc_br_101_t_r = sr_sc_t_r[sr_sc_t_r['br'] == 101]
sr_sc_br_116_t_r = sr_sc_t_r[sr_sc_t_r['br'] == 116]

regionais = teste1['regional'].unique()

#bases de dados que serão utilizadas
df_treinamento_r = [sr_df_r, sr_df_br_020_r, sr_df_br_040_r, sr_sp_r, sr_sp_br_101_r,
                  sr_sp_br_116_r, sr_sc_r, sr_sc_br_101_r, sr_sc_br_116_r]
df_teste_r = [sr_df_t_r, sr_df_br_020_t_r, sr_df_br_040_t_r,sr_sp_t_r, sr_sp_br_101_t_r,
            sr_sp_br_116_t_r, sr_sc_t_r, sr_sc_br_101_t_r, sr_sc_br_116_t_r]

In [8]:
#base de dados para algoritmos ANN, Gradiente Boosting Regressor
#treino anos 2017, 2018, 2019, 2020
#validação 2021
#teste 2022
treinamento2 = pd.concat([df_2017, df_2018, df_2019, df_2020], ignore_index=True)
treinamento2 = treinamento2[colunas]
teste2 = df_2021[colunas]
validacao2 = df_2022[colunas]

#dados para teste
treinamento2 = treinamento2[colunas]
sr_df = treinamento2[treinamento2['regional'] == 'SPRF-DF']
sr_df_br_020 = sr_df[sr_df['br'] == 20]
sr_df_br_040 = sr_df[sr_df['br'] == 40]
sr_sp = treinamento2[treinamento2['regional'] =='SPRF-SP']
sr_sp_br_101 = sr_sp[sr_sp['br'] == 101]
sr_sp_br_116 = sr_sp[sr_sp['br'] == 116]
sr_sc = treinamento2[treinamento2['regional'] =='SPRF-SC']
sr_sc_br_101 = sr_sc[sr_sc['br'] == 101]
sr_sc_br_116 = sr_sc[sr_sc['br'] == 116]

#dados para teste
sr_df_t = teste2[teste2['regional'] == 'SPRF-DF']
sr_df_br_020_t = sr_df_t[sr_df_t['br'] == 20]
sr_df_br_040_t = sr_df_t[sr_df_t['br'] == 40]
sr_sp_t = teste2[teste2['regional'] =='SPRF-SP']
sr_sp_br_101_t = sr_sp_t[sr_sp_t['br'] == 101]
sr_sp_br_116_t = sr_sp_t[sr_sp_t['br'] == 116]
sr_sc_t = teste2[teste2['regional'] =='SPRF-SC']
sr_sc_br_101_t = sr_sc_t[sr_sc_t['br'] == 101]
sr_sc_br_116_t = sr_sc_t[sr_sc_t['br'] == 116]

#dados para validação
sr_df_v = validacao2[validacao2['regional'] == 'SPRF-DF']
sr_df_br_020_v = sr_df_v[sr_df_v['br'] == 20]
sr_df_br_040_v = sr_df_v[sr_df_v['br'] == 40]
sr_sp_v = validacao2[validacao2['regional'] =='SPRF-SP']
sr_sp_br_101_v = sr_sp_v[sr_sp_v['br'] == 101]
sr_sp_br_116_v = sr_sp_v[sr_sp_v['br'] == 116]
sr_sc_v = validacao2[validacao2['regional'] =='SPRF-SC']
sr_sc_br_101_v = sr_sc_v[sr_sc_v['br'] == 101]
sr_sc_br_116_v = sr_sc_v[sr_sc_v['br'] == 116]

regionais = teste2['regional'].unique()

In [9]:
#bases de dados que serão utilizadas
df_treinamento = [sr_df, sr_df_br_020, sr_df_br_040, sr_sp, sr_sp_br_101,
                  sr_sp_br_116, sr_sc, sr_sc_br_101, sr_sc_br_116]
df_validacao =  [sr_df_v, sr_df_br_020_v, sr_df_br_040_v, sr_sp_v, sr_sp_br_101_v,
             sr_sp_br_116_v, sr_sc_v, sr_sc_br_101_v, sr_sc_br_116_v]
df_teste = [sr_df_t, sr_df_br_020_t, sr_df_br_040_t,sr_sp_t, sr_sp_br_101_t,
            sr_sp_br_116_t, sr_sc_t, sr_sc_br_101_t, sr_sc_br_116_t]



# MODELOS DE REGRESSÃO

**Bases de dados que serão utilizadas**

O base de dados de treinamento é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias a partir do ano de 2017 até o ano de 2021
> df_treinamento_r = [sr_df_r, sr_df_br_020_r, sr_df_br_040_r, sr_sp_r, sr_sp_br_101_r,sr_sp_br_116_r, sr_sc_r, sr_sc_br_101_r, sr_sc_br_116_r]

O base de dados de teste é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias no ano de 2022

> df_teste_r = [sr_df_t_r, sr_df_br_020_t_r, sr_df_br_040_t_r,sr_sp_t_r, sr_sp_br_101_t_r, sr_sp_br_116_t_r, sr_sc_t_r, sr_sc_br_101_t_r, sr_sc_br_116_t_r]




---



In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor

def avaliar_modelos(bases_treinamento, bases_teste):
     for i, (base_treinamento, base_teste) in enumerate(zip(bases_treinamento, bases_teste)):
        def imprimir_nome_base(df):
            base_name = [name for name, var in globals().items() if var is df][0]
            print('\n')
            print(f"Executando modelo com a base de treinamento: {base_name}")

        # Separar as características (X) e o alvo (y)
        imprimir_nome_base(base_treinamento)
        X_train = base_treinamento.drop(columns=['km','regional'])
        y_train = base_treinamento['km']

        X_test = base_teste.drop(columns=['km','regional'])
        y_test = base_teste['km']

        # Normalizar os dados
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Modelos
        modelos = {
            "Decision Tree Regressor CV": GridSearchCV(DecisionTreeRegressor(), {}, cv=5),
            "Gradient Boosting Regressor": GradientBoostingRegressor(),
            "Decision Tree Regressor": DecisionTreeRegressor(),
            "Random Forest Regressor CV": GridSearchCV(RandomForestRegressor(), {}, cv=5),
            "Random Forest Regressor": RandomForestRegressor(),
            "XGB Regressor": XGBRegressor(),
            "Elastic Net": ElasticNet(),
            "Lasso": Lasso(),
            "Ridge": Ridge(),
            "SVR com kernel='rbf'": SVR(kernel="rbf"),
            "K-neighbors Regressor": KNeighborsRegressor(),
            "SVR com kernel='poly'": SVR(kernel="poly"),
            "Linear SVR": SVR(kernel="linear"),
            "Linear SVR com CV": GridSearchCV(SVR(kernel="linear"), {}, cv=5),
            "SVR com kernel='linear'": SVR(kernel="linear")
        }

        # Treinamento e avaliação de cada modelo
        for nome_modelo, modelo in modelos.items():
            print(f"\n{nome_modelo}:")

            # Se o modelo for um objeto GridSearchCV, fazemos um ajuste aos dados de treinamento
            if isinstance(modelo, GridSearchCV):
                modelo.fit(X_train_scaled, y_train)
                modelo = modelo.best_estimator_
            else:
                modelo.fit(X_train_scaled, y_train)

            # Fazendo previsões
            y_pred = modelo.predict(X_test_scaled)

            # Calculando o erro médio quadrado (MSE)
            mse = mean_squared_error(y_test, y_pred)

            # Calculando o R² Score
            r2 = r2_score(y_test, y_pred)

            # Calculando o RMSE em quilômetros
            rmse_km = mean_squared_error(y_test, y_pred, squared=False)

            print(f"MSE: {mse}")
            print(f"R² Score: {r2}")
            print(f"RMSE (km): {rmse_km}")

# Exemplo de uso:
avaliar_modelos(df_treinamento_r, df_teste_r)




Executando modelo com a base de treinamento: sr_df_r

Decision Tree Regressor CV:
MSE: 717.2119575557994
R² Score: 0.5206406400920451
RMSE (km): 26.78081323551993

Gradient Boosting Regressor:
MSE: 506.3437089191009
R² Score: 0.6615775941214752
RMSE (km): 22.502082324067274

Decision Tree Regressor:
MSE: 753.4150239358459
R² Score: 0.4964437781409613
RMSE (km): 27.4484065828209

Random Forest Regressor CV:
MSE: 587.7795538969959
R² Score: 0.6071487267400636
RMSE (km): 24.24416535781333

Random Forest Regressor:
MSE: 579.9477621250438
R² Score: 0.6123832221373929
RMSE (km): 24.082104603315795

XGB Regressor:
MSE: 519.2421859029043
R² Score: 0.6529567037339046
RMSE (km): 22.786886270460567

Elastic Net:
MSE: 1488.6590499041217
R² Score: 0.005032415082530939
RMSE (km): 38.58314463472517

Lasso:
MSE: 2364.916990443974
R² Score: -0.5806277110019773
RMSE (km): 48.630412196936746

Ridge:
MSE: 2656.9232135797265
R² Score: -0.7757944462144248
RMSE (km): 51.54535103750605

SVR com kernel='rbf'

#MODELOS UTILIZANDO REDES NEURAIS ARTIFICIAIS



**Bases de dados que serão utilizadas**

O base de dados de treinamento é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias a partir do ano de 2017 até o ano de 2020


> df_treinamento = [sr_df, sr_df_br_020, sr_df_br_040, sr_sp, sr_sp_br_101,
                  sr_sp_br_116, sr_sc, sr_sc_br_101, sr_sc_br_116]



O base de dados de teste é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias no ano de 2021


>df_teste = [sr_df_t, sr_df_br_020_t, sr_df_br_040_t,sr_sp_t, sr_sp_br_101_t,
            sr_sp_br_116_t, sr_sc_t, sr_sc_br_101_t, sr_sc_br_116_t]



O base de dados de validação é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias no ano de 2022


> df_validacao =  [sr_df_v, sr_df_br_020_v, sr_df_br_040_v, sr_sp_v, sr_sp_br_101_v,sr_sp_br_116_v, sr_sc_v, sr_sc_br_101_v, sr_sc_br_116_v]






In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBRegressor

def avaliar_modelos(bases_treinamento, bases_teste, bases_validacao):
    for i, (base_treinamento, base_teste, base_validacao) in enumerate(zip(bases_treinamento, bases_teste, bases_validacao)):
        def imprimir_nome_base(df):
            base_name = [name for name, var in globals().items() if var is df][0]
            print('\n')
            print(f"Executando modelo com a base de treinamento: {base_name}")

        # Separar as características (X) e o alvo (y)
        imprimir_nome_base(base_treinamento)
        X_train = base_treinamento.drop(columns=['km','regional'])
        y_train = base_treinamento['km']

        X_test = base_teste.drop(columns=['km','regional'])
        y_test = base_teste['km']

        X_val = base_validacao.drop(columns=['km','regional'])
        y_val = base_validacao['km']

        # Normalizar os dados
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        X_val_scaled = scaler.transform(X_val)

        # Modelo de rede neural
        modelo = Sequential([
            Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
            Dropout(0.5),
            Dense(64, activation='relu'),
            Dropout(0.5),
            Dense(1)
        ])

        # Compilação do modelo
        modelo.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', 'mae'])

        # Treinamento do modelo
        modelo.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)

        # Avaliação do modelo
        loss, mse, mae = modelo.evaluate(X_val_scaled, y_val)

        # Fazendo previsões
        y_pred = modelo.predict(X_val_scaled)

        # Calculando o erro médio quadrado (MSE)
        mse = mean_squared_error(y_val, y_pred)

        # Calculando o R² Score
        r2 = r2_score(y_val, y_pred)

        # Calculando o RMSE em quilômetros
        rmse_km = mean_squared_error(y_val, y_pred, squared=False)

        print(f"MSE: {mse}")
        print(f"R² Score: {r2}")
        print(f"RMSE (km): {rmse_km}")

# Exemplo de uso:
avaliar_modelos(df_treinamento, df_teste, df_validacao)




Executando modelo com a base de treinamento: sr_df
57/57 [==============================] - 0s 4ms/step
MSE: 411.0874161776497
R² Score: 0.7252435648776785
RMSE (km): 20.275290779114606


Executando modelo com a base de treinamento: sr_df_br_020
15/15 [==============================] - 0s 2ms/step
MSE: 1004.8323482233834
R² Score: 0.6855173055186752
RMSE (km): 31.699090652941187


Executando modelo com a base de treinamento: sr_df_br_040
18/18 [==============================] - 0s 3ms/step
MSE: 242.71845416860756
R² Score: 0.728351564565636
RMSE (km): 15.579424064085538


Executando modelo com a base de treinamento: sr_sp
137/137 [==============================] - 0s 1ms/step
MSE: 18075.369428090155
R² Score: 0.02600217205117339
RMSE (km): 134.44467050831787


Executando modelo com a base de treinamento: sr_sp_br_101
6/6 [==============================] - 0s 2ms/step
MSE: 6994.848289483219
R² Score: -21.204326140894814
RMSE (km): 83.63520962778307


Executando modelo com a base de tr

#Modelo simples de regressão linear

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

def executar_regressao_linear(bases_treinamento, bases_teste):
    for i, (base_treinamento, base_teste) in enumerate(zip(bases_treinamento, bases_teste)):
        def imprimir_nome_base(df):
            base_name = [name for name, var in globals().items() if var is df][0]
            print('\n')
            print(f"Executando modelo com a base de treinamento: {base_name}")

        imprimir_nome_base(base_treinamento)
        # Separar as características (X) e o alvo (y)
        X_train = base_treinamento.drop(columns=['km', 'regional'])
        y_train = base_treinamento['km']

        X_test = base_teste.drop(columns=['km', 'regional'])
        y_test = base_teste['km']

        # Normalizar os dados
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Criar e treinar o modelo de regressão linear
        modelo = LinearRegression()
        modelo.fit(X_train_scaled, y_train)

        # Fazer previsões
        y_pred = modelo.predict(X_test_scaled)

        # Calcular as métricas de avaliação
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        rmse_km = mean_squared_error(y_test, y_pred, squared=False)

        # Imprimir as métricas
        print(f"MSE: {mse}")
        print(f"R² Score: {r2}")
        print(f"RMSE (km): {rmse_km}")

# Função para imprimir o nome da base de dados
def imprimir_nome_base(df):
    base_name = [name for name, var in globals().items() if var is df][0]
    print(f"Base de dados: {base_name}")

# Exemplo de uso:
executar_regressao_linear(df_treinamento_r, df_teste_r)



Executando modelo com a base de treinamento: sr_df_r
MSE: 2660.3625694290754
R² Score: -0.7780931912382223
RMSE (km): 51.578702672993586


Executando modelo com a base de treinamento: sr_df_br_020_r
MSE: 11572.573953876126
R² Score: -2.6218720919306
RMSE (km): 107.57589857340781


Executando modelo com a base de treinamento: sr_df_br_040_r
MSE: 252.73441950804084
R² Score: 0.717141781102151
RMSE (km): 15.897623077304383


Executando modelo com a base de treinamento: sr_sp_r
MSE: 14838.842130786044
R² Score: 0.20040361763227033
RMSE (km): 121.81478617469246


Executando modelo com a base de treinamento: sr_sp_br_101_r
MSE: 17200.03397053566
R² Score: -53.59949209912882
RMSE (km): 131.1488999974291


Executando modelo com a base de treinamento: sr_sp_br_116_r
MSE: 15870.420643229234
R² Score: 0.046852867800308284
RMSE (km): 125.97785774980154


Executando modelo com a base de treinamento: sr_sc_r
MSE: 16920.915883394693
R² Score: 0.07307381078235453
RMSE (km): 130.08042083032592


Exec

#MODELO DE ML DE CLASSIFICAÇÃO

**Bases de dados que serão utilizadas**

O base de dados de treinamento é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias a partir do ano de 2017 até o ano de 2021
> df_treinamento_r = [sr_df_r, sr_df_br_020_r, sr_df_br_040_r, sr_sp_r, sr_sp_br_101_r,sr_sp_br_116_r, sr_sc_r, sr_sc_br_101_r, sr_sc_br_116_r]

O base de dados de teste é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias no ano de 2022

> df_teste_r = [sr_df_t_r, sr_df_br_020_t_r, sr_df_br_040_t_r,sr_sp_t_r, sr_sp_br_101_t_r, sr_sp_br_116_t_r, sr_sc_t_r, sr_sc_br_101_t_r, sr_sc_br_116_t_r]

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def executar_modelos_classificacao(bases_treinamento, bases_teste):
    modelos = {
        "Decision Tree Classifier": DecisionTreeClassifier(),
        "Linear SVC": LinearSVC(max_iter=1000, tol=1e-5),
        "Bagging Classifier": BaggingClassifier(),
    }

    scaler = StandardScaler()  # Criar um objeto scaler uma vez

    for i, (base_treinamento, base_teste) in enumerate(zip(bases_treinamento, bases_teste)):
        print(f"\nExecutando modelos para a base de treinamento {i + 1}:")
        imprimir_nome_base(base_treinamento)

        X_train = base_treinamento.drop(columns=['km', 'regional'])
        y_train = base_treinamento['km']

        X_test = base_teste.drop(columns=['km', 'regional'])
        y_test = base_teste['km']

        # Escalar os dados de treinamento
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)

        # Escalar os dados de teste usando o mesmo scaler
        X_test_scaled = scaler.transform(X_test)

        for nome_modelo, modelo in modelos.items():
            print(f"\n{nome_modelo}:")
            modelo.fit(X_train_scaled, y_train)  # Usar dados de treinamento escalados
            y_pred = modelo.predict(X_test_scaled)  # Usar dados de teste escalados
            accuracy = accuracy_score(y_test, y_pred)
            print(f"Acurácia: {accuracy}")

# Função para imprimir o nome da base de dados
def imprimir_nome_base(df):
    base_name = [name for name, var in globals().items() if var is df][0]
    print(f"Base de dados: {base_name}")

# Exemplo de uso:
executar_modelos_classificacao(df_treinamento_r, df_teste_r)



Executando modelos para a base de treinamento 1:
Base de dados: sr_df_r

Decision Tree Classifier:
Acurácia: 0.04445664105378705

Linear SVC:


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Acurácia: 0.06750823271130625

Bagging Classifier:
Acurácia: 0.05433589462129528

Executando modelos para a base de treinamento 2:
Base de dados: sr_df_br_020_r

Decision Tree Classifier:
Acurácia: 0.029166666666666667

Linear SVC:


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Acurácia: 0.052083333333333336

Bagging Classifier:
Acurácia: 0.03958333333333333

Executando modelos para a base de treinamento 3:
Base de dados: sr_df_br_040_r

Decision Tree Classifier:
Acurácia: 0.0711743772241993

Linear SVC:


...

Acurácia: 0.007627564439768543

Bagging Classifier:
Acurácia: 0.03537611783271962

Executando modelos para a base de treinamento 8:
Base de dados: sr_sc_br_101_r

Decision Tree Classifier:
Acurácia: 0.03891449052739376

Linear SVC:


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Acurácia: 0.06041986687147977

Bagging Classifier:
Acurácia: 0.04045058883768561

Executando modelos para a base de treinamento 9:
Base de dados: sr_sc_br_116_r

Decision Tree Classifier:
Acurácia: 0.05013192612137203

Linear SVC:


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Acurácia: 0.05013192612137203

Bagging Classifier:
Acurácia: 0.04221635883905013


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def executar_modelos_classificacao_2(bases_treinamento, bases_teste):
    modelos = {
        "SVC": SVC(),
        "Random Forest Classifier": RandomForestClassifier(),
        "Logistic Regression": LogisticRegression(solver='saga', max_iter=1000, C=0.1),
        "AdaBoost Classifier": AdaBoostClassifier()
    }

    scaler = StandardScaler()  # Criar um objeto scaler uma vez

    for i, (base_treinamento, base_teste) in enumerate(zip(bases_treinamento, bases_teste)):
        print(f"\nExecutando modelos para a base de treinamento {i + 1}:")
        imprimir_nome_base(base_treinamento)

        X_train = base_treinamento.drop(columns=['km', 'regional'])
        y_train = base_treinamento['km']

        X_test = base_teste.drop(columns=['km', 'regional'])
        y_test = base_teste['km']

        # Escalar os dados de treinamento
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)

        # Escalar os dados de teste usando o mesmo scaler
        X_test_scaled = scaler.transform(X_test)

        for nome_modelo, modelo in modelos.items():
            print(f"\n{nome_modelo}:")
            modelo.fit(X_train_scaled, y_train)  # Usar dados de treinamento escalados
            y_pred = modelo.predict(X_test_scaled)  # Usar dados de teste escalados
            accuracy = accuracy_score(y_test, y_pred)
            print(f"Acurácia: {accuracy}")

# Função para imprimir o nome da base de dados
def imprimir_nome_base(df):
    base_name = [name for name, var in globals().items() if var is df][0]
    print(f"Base de dados: {base_name}")

# Exemplo de uso:
executar_modelos_classificacao_2(df_treinamento_r, df_teste_r)


Executando modelos para a base de treinamento 1:
Base de dados: sr_df_r

SVC:
Acurácia: 0.07354555433589462

Random Forest Classifier:
Acurácia: 0.05378704720087816

Logistic Regression:
Acurácia: 0.05982436882546652

AdaBoost Classifier:
Acurácia: 0.06092206366630077

Executando modelos para a base de treinamento 2:
Base de dados: sr_df_br_020_r

SVC:
Acurácia: 0.0375

Random Forest Classifier:
Acurácia: 0.041666666666666664

Logistic Regression:
Acurácia: 0.020833333333333332

AdaBoost Classifier:
Acurácia: 0.004166666666666667

Executando modelos para a base de treinamento 3:
Base de dados: sr_df_br_040_r

SVC:
Acurácia: 0.08718861209964412

Random Forest Classifier:
Acurácia: 0.07473309608540925

Logistic Regression:
Acurácia: 0.10320284697508897

AdaBoost Classifier:
Acurácia: 0.005338078291814947

Executando modelos para a base de treinamento 4:
Base de dados: sr_sp_r

SVC:
Acurácia: 0.00412560165024066

Random Forest Classifier:
Acurácia: 0.029796011918404766

Logistic Regressi